*If you run this notebook at colab.research.google.com, you need to install packages with the following command*:

In [ ]:
# !pip install -r https://raw.githubusercontent.com/obidam/ds2-2022/main/requirements.txt
# !curl https://raw.githubusercontent.com/obidam/ds2-2022/main/practice/exploratory_statitics/tuto_tools.py --output tuto_tools.py

# Clustering - Tuto 2

Here we play with a 2D dataset and several clustering methods

The 2D dataset is created from a collection of Argo temperature and salinity profiles interpolated on Standard Depth Levels

Clustering methods are: DBSCAN and KMEANS from scikit-learn

(c) G. Maze

In [ ]:
# Libraries import section
import os

import xarray as xr
import numpy as np
# from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context="notebook", style="whitegrid", palette="deep", color_codes=True)

from sklearn import preprocessing
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

from tuto_tools import create_map
from intake import open_catalog

# Load data

From Google cloud storage [see specific tuto here](https://github.com/obidam/ds2-2022/blob/main/practice/environment/02-Access_to_data_in_the_cloud.ipynb).

Use in-situ measurements from Argo floats, interpolated on standard depth levels

In [ ]:
catalog_url = 'https://raw.githubusercontent.com/obidam/ds2-2020/ds2-2021/ds2_data_catalog.yml'
cat = open_catalog(catalog_url)
ds = cat['argo_global_homogeneous_sdl'].read_chunked()
print(ds)

In [ ]:
print('This dataset holds: %.3f GB' % (ds.nbytes / 1e9))

# Define a 2D dataset

## Eg 1: Select temperature and salinity from a single depth level

In [ ]:
z = -250 # Depth level to select
feat_0 = ds['TEMP'].sel(DEPTH=z, method='nearest')
feat_1 = ds['PSAL'].sel(DEPTH=z, method='nearest')

## Eg 2: Select temperature from two distinct depth levels

In [ ]:
feat_0 = ds['TEMP'].sel(DEPTH=0, method='nearest')
feat_1 = ds['TEMP'].sel(DEPTH=-1000, method='nearest')

## Create a new xarray DataSet to hold analysis results:

In [ ]:
RES = xr.concat([feat_0, feat_1], dim='N_FEATURES').chunk(chunks={'N_PROF':1000})
RES = RES.rename('ARGO')
# Remove obsolete attributes:
for a in RES.attrs.copy(): 
    del RES.attrs[a]
# Add feature names:
RES.attrs['feature_0'] = "%s at %0.2fm depth"%(feat_0.name,feat_0['DEPTH'])
RES.attrs['feature_1'] = "%s at %0.2fm depth"%(feat_1.name,feat_1['DEPTH'])
# We're good:
print(RES)

### Possibly sub-sample the array for faster computation

In [ ]:
n = 10000 # Nb of sample to keep
# print(len(RES.N_PROF))
ii = np.random.choice(range(len(RES.N_PROF)), n, replace=False)
RES = RES.isel(N_PROF=ii).chunk({'N_PROF':n/10})
print(RES)

# Dataset PDF

## Marginal PDFs

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=plt.figaspect(0.5), dpi=120, facecolor='w', edgecolor='k')
sns.histplot(RES.isel(N_FEATURES=0), stat='density', kde=True, ax=ax[0])
sns.histplot(RES.isel(N_FEATURES=1), stat='density', kde=True, ax=ax[1])
ax[0].set_xlabel(RES.attrs['feature_0']);
ax[1].set_xlabel(RES.attrs['feature_1']);

## Marginal and 2D PDFs:

In [ ]:
g = (sns.jointplot(x=RES.isel(N_FEATURES=0), y=RES.isel(N_FEATURES=1), kind="kde", color="k")
     .set_axis_labels(RES.attrs['feature_0'], RES.attrs['feature_1'])
     .plot_joint(sns.kdeplot, zorder=0, n_levels=6))

## Exercise

Plot conditional PDFs: p(x1|x2)

# Clustering in 2D

Clearly from the figure above, one can see that the dataset exhibits several modes. 
In other words, data samples agregated into several clusters.

Let's identify them

## Prepare data (normalisation)

In [ ]:
# Scaler
X = RES.values.T 
print(X.shape) # Ensure we have sampling x features

# Fit the scaler object:
scaler = preprocessing.StandardScaler()
%time scaler = scaler.fit(X)

# The mean and std profiles are in the scaler object properties:
X_ave = scaler.mean_
X_std = scaler.scale_
print("Data mean and std:", X_ave, X_std)

# Normalize data:
Xn = scaler.transform(X) 
# print(Xn.shape)

# Add results to the dataset:
RES['TEMPnorm'] = xr.DataArray(Xn[:,0], dims='N_PROF')
RES['PSALnorm'] = xr.DataArray(Xn[:,1], dims='N_PROF')

## Clustering with DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.1, min_samples=50).fit(Xn)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)

# Add results to the dataset:
RES['DBSCAN'] = xr.DataArray(labels, dims='N_PROF')

print(RES)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=plt.figaspect(0.6), dpi=90, facecolor='w', edgecolor='k')

# 
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

plt.plot(Xn[:,1],Xn[:,0],'.',markersize=1,color=[0.3]*3)

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 0.2]
#     print "Cluster: ", k, col
    
    class_member_mask = (labels == k)

    xy = Xn[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=tuple(col),
             markeredgecolor=[0.5]*3, markersize=3, markeredgewidth=0.5, label="Class %i"%(k))

#     xy = Xn[class_member_mask & ~core_samples_mask]
#     plt.plot(xy[:, 1], xy[:, 0], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='none', markersize=3, label="Noise class %i"%(k))

plt.xlim(-3,9)
plt.ylim(-3,3)
plt.xlabel(RES.attrs['feature_0'])
plt.ylabel(RES.attrs['feature_1'])
plt.title('Estimated number of clusters: %d (DBSCAN)' % n_clusters_)
plt.legend()
plt.show()

In [ ]:
fig, proj, ax = create_map(extent=[-180, 180, -90, 90])

# plt.scatter(RES.LONGITUDE,RES.LATITUDE,5,RES.DBSCAN,transform=proj)

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 0.2]
#     print "Cluster: ", k, col
    
    class_member_mask = (RES.DBSCAN == k)

    plt.plot(RES.LONGITUDE[class_member_mask],
                RES.LATITUDE[class_member_mask],
                 'o', markerfacecolor=tuple(col),
                 markeredgecolor='none', markersize=2,  
             label="Class %i"%(k), transform=proj)

plt.title('Estimated number of clusters: %d (DBSCAN)' % n_clusters_)
# plt.legend()
plt.show()

## Clustering with KMeans

In [ ]:
from sklearn.cluster import KMeans

%time kmeans = KMeans(n_clusters=6, random_state=0).fit(Xn)
labels = kmeans.predict(Xn)
n_clusters_ = kmeans.n_clusters
kmeans.cluster_centers_.shape

# Add results to the dataset:
RES['KMEANS'] = xr.DataArray(labels, dims='N_PROF')

print(RES)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=plt.figaspect(0.6), dpi=80, facecolor='w', edgecolor='k')

unique_labels = np.unique(RES['KMEANS'])
n_clusters_ = unique_labels.shape[0]
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

for k, col in zip(range(n_clusters_), colors):
#     print "Cluster: ", k, col
    class_members = RES['KMEANS'] == k
#     plt.plot(RES['PSALnorm'][class_members], RES['TEMPnorm'][class_members], '.', 
#              markerfacecolor=tuple(col), markeredgecolor='none')

#     cluster_center = kmeans.cluster_centers_[k,:]
#     plt.plot(cluster_center[1], cluster_center[0], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=14, label='Class %i'%(k))
    
    plt.plot(RES.sel(N_FEATURES=1)[class_members], RES.sel(N_FEATURES=0)[class_members], '.', 
             markerfacecolor=tuple(col), markeredgecolor='none', label='Class %i'%(k))

    cluster_center = scaler.inverse_transform(kmeans.cluster_centers_[k,:][np.newaxis,:])[0]
    plt.plot(cluster_center[1], cluster_center[0], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=10, label='Class center #%i'%(k))

# plt.xlim(-3,9)
# plt.ylim(-3,3)
plt.xlabel(RES.attrs['feature_0'])
plt.ylabel(RES.attrs['feature_1'])
plt.title('Number of clusters: %i (KMEANS)' % n_clusters_)
plt.legend()
plt.show()

In [ ]:
fig, proj, ax = create_map(extent=[-180, 180, -90, 90])

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 0.2]
    print("Cluster: ", k, col)
    
    class_member_mask = (RES.KMEANS == k)

    plt.plot(RES.LONGITUDE[class_member_mask],
                RES.LATITUDE[class_member_mask],
                 'o', markerfacecolor=tuple(col),
                 markeredgecolor='none', markersize=2,  
             label="Class %i"%(k), transform=proj)

plt.title('Number of clusters: %d (KMEANS)' % n_clusters_)
# plt.legend()
plt.show()

## Exercise

Implement and visualize results from another clusterring method available in scikit-learn